In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import json
train_data=pd.read_csv("Finetune_data_75.csv",encoding='latin1')

In [2]:
train_data.head()

,Indicators (Select from drop-down list),Description Simplified,Geographic Coverage,Format,Spatial Resolution,Temporal Resolution,Temporal Extent,Latency,Source/Link,Project,Data Visualization
0,Disasters,NASA's Black Marble night lights data provide ...,"Puerto Rico, New Orleans","tif, jpeg, mpeg",NaN,NaN,2017,NaN,https://svs.gsfc.nasa.gov/4658#:~:text=NASA%27...,Disaster Dashboard,True
1,Urban Flooding,The Flood Dashboard brings together mutliple N...,Global,Interactive Visualization,Varies,Varies,Varies,1 Day,https://maps.disasters.nasa.gov/arcgis/apps/op...,NaN,True
2,Human Dimensions,The Global Cyclone Proportional Economic Loss ...,Global,"ASCII, PDF, PNG, WMS",0.0417 degrees,NaN,2000-01-01 to 2000-12-31,NaN,https://doi.org/10.7927/H44F1NNF,NDH - Natural Disaster Hotspots,True
3,Food Availability,The Global Food Security-support Analysis Data...,Africa,GeoTIFF,30m,NaN,2013-01-01 to 2016-06-30,NaN,https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30...,MEaSUREs - Making Earth System Data Records fo...,False
4,Climate Change,The MuSLI Multi-Source Land Surface Phenology ...,North America,netCDF-4,30m,NaN,2016-01-01 to 2019-12-31,NaN,https://doi.org/10.5067/Community/MuSLI/MSLSP3...,NaN,False


In [3]:
final_data={}
for each_data in train_data.iterrows():
    final_data[each_data[1][8]]={"Indicators":each_data[1][0],
              "Description":each_data[1][1],
              "Geographic_Coverage":each_data[1][2],
              "Format":each_data[1][3],
              "Spatial_Resolution":each_data[1][4],
              "Temporal_Resolution":each_data[1][5],
              "Temporal_Extent":each_data[1][6],
              "Latency":each_data[1][7],
                "Project":each_data[1][9],
                "Data_Visualization":each_data[1][10]}
    

In [4]:
for url,value in final_data.items():
    response=requests.get(url)
    html_page = response.text
    soup = BeautifulSoup(html_page, "html.parser")
    text=soup.get_text()
    text = re.sub(r'[\t\n\r\f\v]+', '', text)
    text=re.sub(r'[^\w\s]', '', text)
    final_data[url]["text"]=text


In [9]:
_SYSTEM_PROMPT ="Extract metadata and entities details accurately from my requests." + "Metadata extraction for Indicators should clearly be one of the element in the list ['Disasters','Human Dimensions','Food Availability','Health & Air Quality','Water Availability', 'Extreme Heat','Urban Flooding','Climate']. Do not create new element for indicators " + "Make sure the following fields follow the following regex pattern"+"Spatial_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$"+"Temporal_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$"+"Temporal_Extent: ^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$"


In [14]:
train_data=[]
for url,value in final_data.items():
#     data={"messages": [{"role": "system", "content":_SYSTEM_PROMPT},{"role": "user", "content": f'{value["text"]}'}, {"role": "assistant", "content": {"Indicators":value["Indicators"],"Description":value["Description"],"Geographic_Coverage":value["Geographic_Coverage"], "Format":value["Format"],"Spatial_Resolution":value["Spatial_Resolution"],"Temporal_Resolution":value["Temporal_Resolution"],"Temporal_Extent":value["Temporal_Extent"],"Latency":value["Latency"],"Project":value["Project"],"Data_Visualization":value["Data_Visualization"]}}]}
    output_content={"Indicators":value["Indicators"],"Description":value["Description"],"Geographic_Coverage":value["Geographic_Coverage"], "Format":value["Format"],"Spatial_Resolution":value["Spatial_Resolution"],"Temporal_Resolution":value["Temporal_Resolution"],"Temporal_Extent":value["Temporal_Extent"],"Latency":value["Latency"],"Project":value["Project"],"Data_Visualization":value["Data_Visualization"]}
    output_content=str(output_content)
    data={"messages": [{"role": "system", "content":_SYSTEM_PROMPT},{"role": "user", "content":f'{value["text"]}'}, {"role": "assistant", "content": output_content}]}
    train_data.append(data)
    
with open("train_data_75.jsonl", 'w') as jsonl_file:
    for record in train_data:
        jsonl_file.write(json.dumps(record) + '\n')



In [3]:
import pickle
train_75_urls=list(final_data.keys())
with open("train_75_urls.pkl",'wb') as file:
    pickle.dump(train_75_urls,file)


NameError: name 'final_data' is not defined

In [11]:
# !pip install -e .
# !pip install spacy
# !python3 -m spacy download en_core_web_lg
# !pip install pandas
# !pip install openpyxl
!pip install openai


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [19]:
urls=train_data[0]
print(urls)

{'messages': [{'role': 'system', 'content': "Extract metadata and entities details accurately from my requests.Metadata extraction for Indicators should clearly be one of the element in the list ['Disasters','Human Dimensions','Food Availability','Health & Air Quality','Water Availability', 'Extreme Heat','Urban Flooding','Climate']. Do not create new element for indicators Make sure the following fields follow the following regex patternSpatial_Resolution: ^(\\d+(\\.\\d+)? [a-zA-Z]+|varies|N/A)$Temporal_Resolution: ^(\\d+(\\.\\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$Temporal_Extent: ^(\\d{4}-\\d{2}-\\d{2} to present|present|\\d{4}-\\d{2}-\\d{2} (?:to|until) present|\\d{4}-\\d{2}-\\d{2} to \\d{4}-\\d{2}-\\d{2}|varies(?:- multiple datasets (?:included|available))?)$"}, {'role': 'user', 'content': 'NASA SVS  NASAs Black Marble night lights used to examine disaster rec

In [12]:
!pip install -U openai

  Using cached openai-1.12.0-py3-none-any.whl (226 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Uninstalling openai-0.28.1:
      Successfully uninstalled openai-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
larch 0.0.2a0 requires openai==0.28.1, but you have openai 1.12.0 which is incompatible.
instructor 0.2.9 requires openai<0.29.0,>=0.28.0, but you have openai 1.12.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [15]:
#30, 50, 100,115
from openai import OpenAI
client = OpenAI(api_key="")

client.files.create(
  file=open("train_data_75.jsonl", "rb"),
  purpose="fine-tune"
)


FileObject(id='file-oWdHEhBksMYFzQnCm6b9GRDO', bytes=503427, created_at=1707857941, filename='train_data_75.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [16]:
client.fine_tuning.jobs.create(
  training_file="file-oWdHEhBksMYFzQnCm6b9GRDO", 
  model="gpt-3.5-turbo-1106", 
  hyperparameters={
    "n_epochs":5
  }
)

FineTuningJob(id='ftjob-wFdniQASHY6qvNiJBl2G0Inf', created_at=1707857995, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-KVpMEM9Q8Xm2pc8YuAMYw8r3', result_files=[], status='validating_files', trained_tokens=None, training_file='file-oWdHEhBksMYFzQnCm6b9GRDO', validation_file=None)

In [23]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-wFdniQASHY6qvNiJBl2G0Inf', created_at=1707857995, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-KVpMEM9Q8Xm2pc8YuAMYw8r3', result_files=[], status='running', trained_tokens=None, training_file='file-oWdHEhBksMYFzQnCm6b9GRDO', validation_file=None)], object='list', has_more=True)

In [28]:
client.fine_tuning.jobs.retrieve("ftjob-wFdniQASHY6qvNiJBl2G0Inf")

FineTuningJob(id='ftjob-wFdniQASHY6qvNiJBl2G0Inf', created_at=1707857995, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-1106:nasa::8ruRKvS8', finished_at=1707858978, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-KVpMEM9Q8Xm2pc8YuAMYw8r3', result_files=['file-bNqvaX3huXgc1rT9wo7Rkyfe'], status='succeeded', trained_tokens=550275, training_file='file-oWdHEhBksMYFzQnCm6b9GRDO', validation_file=None)

In [1]:
!pip install -e .
!pip install spacy
!python3 -m spacy download en_core_web_lg
!pip install pandas
!pip install openpyxl
import pickle
import json

Obtaining file:///Users/rajashreedahal/Desktop/Metadata_Finetuning/larch
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done


  Building editable for larch (pyproject.toml) ... done
  Created wheel for larch: filename=larch-0.0.2a0-0.editable-py3-none-any.whl size=14420 sha256=6e6c884475c9a4e21730d68a7ea05e4335fc220f8fe040bf7e84026e08d741ba
  Stored in directory: /private/var/folders/m4/9_ydsvdn3b3d84rntrrsxm0h0000gn/T/pip-ephem-wheel-cache-iarmtz3g/wheels/3a/98/39/46f8f1efc02842106044493327f82ab1f2c43a0a7f389a1df9
Successfully built larch
  Attempting uninstall: larch
    Found existing installation: larch 0.0.2a0
    Uninstalling larch-0.0.2a0:
      Successfully uninstalled larch-0.0.2a0

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.7.1/en_core_web_lg-3.7.1-py3-none-any.whl (587.7 MB)

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
with open("train_data/train_75_urls.pkl",'rb') as file:
    train_75_urls=pickle.load(file)

train_75_urls=[i.strip() for i in train_75_urls]
train_75_urls=list(set(train_75_urls))


In [5]:
train_75_urls

['https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AFCE.001',
 'https://dx.doi.org/10.5067/HLS/HLSS30.002',
 'https://doi.org/10.7927/H4ZK5DQS',
 'https://ldas.gsfc.nasa.gov/fldas',
 'https://search.earthdata.nasa.gov/search?q=AIRS3ST&fi=AIRS&fs10=Surface%20Temperature&fsm0=Atmospheric%20Temperature&fs20=Air%20Temperature&fst0=Atmosphere',
 'https://doi.org/10.7927/H4JW8BTT',
 'https://dx.doi.org/10.5067/ECTSM-MSL44',
 'https://dx.doi.org/10.5067/GMSLT-FJPL1',
 'https://fluid.nccs.nasa.gov/wxmaps/',
 'https://doi.org/10.5067/T4OW83T8EXDO',
 'https://dx.doi.org/10.5067/HLS/HLSL30.002',
 'https://doi.org/10.5067/MOQOVNHNERGG',
 'https://arthurhou.pps.eosdis.nasa.gov/',
 'https://dx.doi.org/10.5067/MEASURES/SO2/DATA205',
 'https://appliedsciences.nasa.gov/our-impact/news/aria-damage-proxy-map-puerto-rico-after-hurricane-maria https://ghis.maps.arcgis.com/home/item.html?id=1ce3ccaacc6c4cd7b3b6cef4ea4980aa',
 'https://nasa.maps.arcgis.com/home/webmap/viewer.html?webmap=f33be724f04b4b4c942edd0c

In [6]:
import pandas as pd
from larch.metadata import InstructorBasedOpenAIMetadataExtractor
from larch.metadata.validators import WhitelistBasedMetadataValidator
from larch.processors import PIIRemover, TextProcessingPipeline
from larch.utils import load_whitelist

text_processor = TextProcessingPipeline(
    lambda x: re.sub(r"\$(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\)(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\#(?=\w|\n|\()", " ", x).strip(),
    lambda x: x.replace("\t", " ").replace("!", " ").strip(),
    PIIRemover()
)
import re
import requests
from bs4 import BeautifulSoup

from pydantic import BaseModel,constr,Field
from typing import Literal

class Environmental_Justice(BaseModel):
    Indicators: Literal["Disasters","Human Dimensions","Food Availability","Health & Air Quality","Water Availability", "Extreme Heat","Urban Flooding","Climate"]
    Description: str
    Geographic_Coverage: str =Field(pattern=r'')
    Format: str
    Spatial_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$')
    Temporal_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$')
    Temporal_Extent: str = Field(pattern=r'^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$')
    Latency: str = Field(pattern=r'^\d+(\s*(Day|days|months|NRT))?$')
    Project: str
    Data_Visualization: bool


    
schema = Environmental_Justice
sheet_names=["Water Availability - Cleaned", "Disasters (Disaster Recovery) -", 
 "Food Availability - Cleaned", "Human Dimensions - Cleaned", "Urban Flooding - Cleaned",
 "Extreme Heat - CIP","Climate (Climate Change) - CIP","Health and Air Quality - CIP"]
used_urls=[]

for each_sheet in sheet_names:
    mapper = pd.read_excel("nasa_esds.xlsx", sheet_name=[each_sheet])
    res = {}
    url_lists=[]
    for field_key, value_df in mapper.items():
        res[field_key] = {}
        cols = value_df.columns
        for i, row in value_df.iterrows():
            source_link_col = [col for col in cols if col == "Source/Link"]
            alternate_vals = filter(None, row[source_link_col])
            alternate_vals=list(alternate_vals)
            if alternate_vals and str(alternate_vals[0]).startswith("http"):
                url_lists.append(alternate_vals[0])
    url_content=[]
    urls=[]
    count=0
    for url in url_lists:
        response=requests.get(url)
        html_page = response.text
        soup = BeautifulSoup(html_page, "html.parser")
        text=soup.get_text()
        text = re.sub(r'[\t\n\r\f\v]+', '', text)
        text=re.sub(r'[^\w\s]', '', text)
        url=url.strip()
        if url not in used_urls and url not in train_75_urls:
            url_content.append(text)
            urls.append(url)
        used_urls.append(url)


    import json
    print(len(url_content))
    metadata_extractor = InstructorBasedOpenAIMetadataExtractor(
    model="ft:gpt-3.5-turbo-1106:nasa::8ruRKvS8",
    schema=schema,
    preprocessor=text_processor,
    debug=True,
)

    response_result={}
    for enum,text in enumerate(url_content):
        split=text.split()
        if len(split)>1900:
            text=" ".join(split[0:1800])
        metadata = metadata_extractor(text)
        response=json.loads((metadata['choices'][0]['message']["function_call"]["arguments"]))
        response_result[urls[enum]]=response
        print(enum,len(text.split()))
    with open("finetuned_extractions/gpt3.5_75/"+each_sheet+".json",'w') as file:
            json.dump(response_result,file,indent=4)

spacy is loaded


2024-02-13 15:23:31.854 | DEBUG    | larch.metadata.extractors_openai:_extract:79 - nchars=8181
Text :: LP DAAC  GFSAD30EUCEARUMECEHomeAboutAbout LP DAAC                                                                                                    News Archive                                                                                                    DataGet Started with Data                                                                                                    Search Data Catalog                                                                                                    Data Citations and Policies                                                                                                    View All DataToolsAppEEARSData PoolData Prep ScriptsNASA Earthdata SearchUSGS EarthExplorerView All ToolsResourcesData in Action                                                                                                    ELearning                           

1
The schema is <class '__main__.Environmental_Justice'>
the system prompt is Extract metadata and entities details accurately from my requests.Metadata extraction for Indicators should clearly be one of the element in the list [“Disasters”,“Human Dimensions”,“Food Availability”,“Health & Air Quality”,“Water Availability”, “Extreme Heat”,“Urban Flooding”,“Climate]. Do not create new element for indicators Make sure the following fields follow the following regex patternSpatial_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$Temporal_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$Temporal_Extent: ^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details ac

0 548


2024-02-13 15:23:58.828 | DEBUG    | larch.metadata.extractors_openai:_extract:79 - nchars=5971
Text :: Global Multihazard Proportional Economic Loss Risk Deciles v1 Natural Disaster Hotspots  SEDAC Socioeconomic Data and Applications Center sedacA Data Center in NASAs Earth Observing System Data and Information System EOSDIS  Hosted by CIESIN at Columbia UniversitySearch SEDACDataData SetsMapsWeb Pagessearch Data Data Sets Data Collections Featured Data Uses Data Citations Citations Database Data SubmissionMaps Map Gallery Map Viewer Map Services Mapping ToolsThemes Agriculture Climate Conservation Governance Hazards Health Infrastructure Land Use Marine and Coastal Population Poverty Remote Sensing Sustainability Urban WaterResources Guides Multimedia Networks News Publications Related Sites Remote Sensing ToolsSocial Media Twitter FaceBook YouTube Flickr Blog Posts CommunitiesAbout About SEDAC User Working Group Privacy User RegistrationHelp Natural Disaster Hotspots  Follow Us Twit

...

25 2
the system prompt is Extract metadata and entities details accurately from my requests.Metadata extraction for Indicators should clearly be one of the element in the list [“Disasters”,“Human Dimensions”,“Food Availability”,“Health & Air Quality”,“Water Availability”, “Extreme Heat”,“Urban Flooding”,“Climate]. Do not create new element for indicators Make sure the following fields follow the following regex patternSpatial_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$Temporal_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$Temporal_Extent: ^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.Metadata extraction for In

2024-02-13 15:29:30.950 | DEBUG    | larch.metadata.extractors_openai:_extract:79 - nchars=8
Text :: GES DISC


26 858
the system prompt is Extract metadata and entities details accurately from my requests.Metadata extraction for Indicators should clearly be one of the element in the list [“Disasters”,“Human Dimensions”,“Food Availability”,“Health & Air Quality”,“Water Availability”, “Extreme Heat”,“Urban Flooding”,“Climate]. Do not create new element for indicators Make sure the following fields follow the following regex patternSpatial_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$Temporal_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$Temporal_Extent: ^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.Metadata extraction for 

2024-02-13 15:29:33.230 | DEBUG    | larch.metadata.extractors_openai:_extract:79 - nchars=8
Text :: GES DISC


27 2
the system prompt is Extract metadata and entities details accurately from my requests.Metadata extraction for Indicators should clearly be one of the element in the list [“Disasters”,“Human Dimensions”,“Food Availability”,“Health & Air Quality”,“Water Availability”, “Extreme Heat”,“Urban Flooding”,“Climate]. Do not create new element for indicators Make sure the following fields follow the following regex patternSpatial_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$Temporal_Resolution: ^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$Temporal_Extent: ^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.Metadata extraction for In

In [7]:
# !pip install sentence_transformers
from difflib import SequenceMatcher
import numpy as np
import re 
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim


encoder = SentenceTransformer('all-mpnet-base-v2').to('cpu')

# Evaluation Metric: Exact Match  #Indicators, temporal resolution, temporal extent,
def evaluate_exact_match(predicted, ground_truth):
    return int(predicted.lower() == ground_truth.lower())

# Evaluation Metric: description, project, geographic measure
def evaluate_similarity_or_edit_distance(predicted, ground_truth1,ground_truth2,encoder):
    if (type(ground_truth1)==float and type(predicted)==float) or (type(ground_truth2)==float and type(predicted)==float):
        return 1
    elif type(ground_truth1)==float or type(ground_truth2)==float or type(predicted)==float:
        return 0
        
    with torch.no_grad():
        pred_embeddings = encoder.encode(predicted, show_progress_bar=False)
        truth1_embeddings = encoder.encode(ground_truth1, show_progress_bar=False)
        truth2_embeddings = encoder.encode(ground_truth2, show_progress_bar=False)
    similarity1 = cos_sim(pred_embeddings, truth1_embeddings).numpy()[0][0]
    similarity2 = cos_sim(pred_embeddings, truth2_embeddings).numpy() [0][0]
    similarity=max(similarity1,similarity2)
    if similarity>0.7:
        return 1
    else:
        return 0
    
def evaluate_project(predicted, ground_truth,encoder):
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    elif type(ground_truth)==float:
        return 0
    ground_truth=ground_truth.split("-",1)
    with torch.no_grad():
        pred_embeddings = encoder.encode(predicted, show_progress_bar=False)
        truth1_embeddings = encoder.encode(ground_truth[0], show_progress_bar=False)
        similarity = cos_sim(pred_embeddings, truth1_embeddings).numpy()[0][0]
        if len(ground_truth)>1:
            truth2_embeddings = encoder.encode(ground_truth[1], show_progress_bar=False)
            similarity1 = cos_sim(pred_embeddings, truth2_embeddings).numpy()[0][0]
            similarity=max(similarity,similarity1)
    if similarity>0.55:
        return 1
    else:
        return 0


# Evaluation Metric: Exact Match or at least one match if multiple
def evaluate_format(predicted, ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    elif type(predicted)==float or type(ground_truth)==float:
        return 0
    predicted=predicted.lower()
    ground_truth=ground_truth.lower()
    if predicted==ground_truth:
        return 1
    predicted_formats = set(predicted.split(","))
    ground_truth_formats = set(ground_truth.split(","))
    if predicted_formats.intersection(ground_truth_formats):
        return 1
    return 0

def evaluate_spatial_resolution(predicted, ground_truth1,ground_truth2):
    predicted = np.nan if predicted=="N/A" else predicted
    if (type(predicted)==float and type(ground_truth1)==float) or (type(predicted)==float and type(ground_truth2)==float):
        return 1
    elif (type(ground_truth1)==float or type(ground_truth2)==float):
        return 0
    elif type(predicted)==float:
        return 0
    predicted=predicted.lower()
    ground_truth1=ground_truth1.lower()
    ground_truth2=ground_truth2.lower()
    if (predicted==ground_truth1) or (predicted==ground_truth2):
        return 1
    predicted = re.sub(r'[a-zA-Z\(\)\[\]]', '', predicted).strip()
    ground_truth1=re.sub(r'[a-zA-Z\(\)\[\]]', '', ground_truth1).strip()
    ground_truth2=re.sub(r'[a-zA-Z\(\)\[\]]', '', ground_truth2).strip()
    if not (ground_truth1 or ground_truth2) and predicted.find(',') != -1:
        return 1
    ground_truth1=re.sub(r'[^\d.]', '', ground_truth1)
    ground_truth2=re.sub(r'[^\d.]', '', ground_truth2)
    predicted=re.sub(r'[^\d.]', '', predicted)
    if predicted==ground_truth1 or predicted==ground_truth2:
        return 1
    if predicted and ((ground_truth1 and (predicted.replace(" ","")==ground_truth1.replace(" ",""))) or (ground_truth2 and (predicted.replace(" ","")==ground_truth2.replace(" ","")))):
        return 1
    return 0


def evaluate_latency(predicted, ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    if type(predicted)==float or type(ground_truth)==float:
        return 0
    predicted=predicted.lower()
    ground_truth=ground_truth.lower()
    if (predicted==ground_truth):
        return 1
    predicted = re.sub(r'[a-zA-Z]', '', predicted)
    predicted=re.sub(r'[^a-zA-Z0-9]','',predicted)
    ground_truth=re.sub(r'[a-zA-Z]', '', ground_truth)
    ground_truth=re.sub(r'[^a-zA-Z0-9]','',ground_truth)
    if not ground_truth and not predicted:
        return 1
    if not ground_truth or not predicted:
        return 0
    if (int(predicted)==int(ground_truth)):
        return 1
    return 0


def evaluate_temporal_resolution_extent(predicted,ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    ground_truth=np.nan if (type(ground_truth)==str and ground_truth.lower()=="nan") else ground_truth
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    if type(predicted)==float or type(ground_truth)==float:
        return 0
    predicted=predicted.lower()
    ground_truth=str(ground_truth).lower()
    if predicted==ground_truth:
        return 1
    predicted = ''.join(re.findall(r'\w', predicted))
    ground_truth=''.join(re.findall(r'\w', ground_truth))
    if predicted==ground_truth:
        return 1 
    return 0

def evaluate_data_visualization_indicators(predicted,ground_truth):
    if predicted==ground_truth:
        return 1
    return 0
    


In [8]:

sheet_names=["Water Availability - Cleaned", "Disasters (Disaster Recovery) -", 
 "Food Availability - Cleaned", "Human Dimensions - Cleaned", "Urban Flooding - Cleaned",
 "Extreme Heat - CIP","Health and Air Quality - CIP", "Climate (Climate Change) - CIP"]

import json
before_finetuned={}
for sheet_name in sheet_names:
    with open(f"extractions/gpt3.5_1106/{sheet_name}.json",'r') as file:
        data=json.load(file)
    before_finetuned.update(data)

after_finetuned={}   
for sheet_name in sheet_names:
    with open(f"finetuned_extractions/gpt3.5_75/{sheet_name}.json",'r') as file:
        data=json.load(file)
    after_finetuned.update(data)


true_data = {}
repeated_rows=[]
for sheet_name in sheet_names:
    mapper = pd.read_excel("nasa_esds.xlsx", sheet_name=[sheet_name])
    for field_key, value_df in mapper.items():
        cols = value_df.columns
        for i, row in value_df.iterrows():
            source_link_col = ["Source/Link","Indicators                     (Select from drop-down list)","Description","Description Simplified","Geographic Coverage","Format","Spatial Resolution","Spatial Resolution (standard)","Temporal Resolution","Temporal Extent","Latency","Project","Data Visualization"]
            alternate_vals=row[source_link_col]
            alternate_vals=list(alternate_vals)
            source_link_col[1]="Indicators"
            alternate_values=alternate_vals[1:]
            value={}
            for enum,i in enumerate(source_link_col[1:]):
                if i=="Data Visualization":
                    if isinstance(alternate_values[enum],str):
                        value[i]=True
                    else:
                        value[i]=False
                else:
                    value[i]=alternate_values[enum]
            if alternate_vals[0] in true_data.keys():
                repeated_rows.append(alternate_vals[0])
            if alternate_vals[0] not in [np.nan, 'DBF', 'Facebook Data For Good High Resolution Population Density Maps']:
                true_data[alternate_vals[0].strip()]=value

    


In [9]:
intersection_keys = set(before_finetuned.keys()) & set(after_finetuned.keys()) & set(true_data.keys())
before_finetuned={k:before_finetuned[k] for k in intersection_keys}
after_finetuned={k:after_finetuned[k] for k in intersection_keys}
true_data={k:true_data[k] for k in intersection_keys}

In [10]:
len(true_data)

72

In [11]:
def boolean_metrics(true_data,pred_data):
    metrics={}
    for key in true_data.keys():
        reject_key=[np.nan, 'DBF', 'Facebook Data For Good High Resolution Population Density Maps']
        if key not in reject_key:
            true_source=true_data[key]
            try:
                pred_source=pred_data[key]
            except:
                pred_source=pred_data[key.strip()]
            for each_key in pred_source.keys():
                if each_key=="Indicators":
                    result=evaluate_exact_match(pred_source[each_key],true_source[each_key])
                    if each_key not in metrics.keys():
                            metrics[each_key]=[result]
                    else:
                        metrics[each_key].append(result)
                elif each_key=="Description":
                    result=evaluate_similarity_or_edit_distance(pred_source[each_key], true_source[each_key],true_source["Description Simplified"],encoder)
                    if each_key not in metrics.keys():
                        metrics[each_key]=[result]
                    else:
                        metrics[each_key].append(result)
                elif each_key=="Geographic_Coverage":
                    evaluate_format(pred_source[each_key],true_source["Geographic Coverage"])
                    if each_key not in metrics.keys():
                        metrics[each_key]=[result]
                    else:
                        metrics[each_key].append(result) 
                elif each_key=="Format":
                    result=evaluate_format(pred_source[each_key],true_source[each_key])
                    if each_key not in metrics.keys():
                        metrics[each_key]=[result]
                    else:
                        metrics[each_key].append(result)
                elif each_key=="Spatial_Resolution":
                    result=evaluate_spatial_resolution(pred_source[each_key],true_source["Spatial Resolution"],true_source["Spatial Resolution (standard)"])
                    if each_key not in metrics.keys():
                        metrics[each_key]=[result]
                    else:
                        metrics[each_key].append(result)
                elif each_key=="Temporal_Resolution":
                    result=evaluate_temporal_resolution_extent(pred_source[each_key],true_source["Temporal Resolution"])
                    if each_key not in metrics.keys():
                        metrics[each_key]=[result]
                    else:
                        metrics[each_key].append(result)
                elif each_key=="Temporal_Extent":
                    result=evaluate_temporal_resolution_extent(pred_source[each_key],true_source["Temporal Extent"])
                    if each_key not in metrics.keys():
                        metrics[each_key]=[result]
                    else:
                        metrics[each_key].append(result)
                elif each_key=="Latency":
                    result=evaluate_latency(pred_source[each_key],true_source["Latency"])
                    if each_key not in metrics.keys():
                        metrics[each_key]=[result]
                    else:
                        metrics[each_key].append(result)
                elif each_key=="Project":
                    result=evaluate_project(pred_source[each_key], true_source[each_key],encoder)
                    if each_key not in metrics.keys():
                        metrics[each_key]=[result]
                    else:
                        metrics[each_key].append(result)
                elif each_key=="Data_Visualization":
                    result=evaluate_data_visualization_indicators(pred_source[each_key], true_source["Data Visualization"])
                    if each_key not in metrics.keys():
                        metrics[each_key]=[result]
                    else:
                        metrics[each_key].append(result)
    return metrics

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def print_metrics(metrics,mode):
    print(f"The metrics {mode} finetuning is:")
    predictions=metrics
    aspect_metrics = {}
    for aspect, preds in predictions.items():
        accuracy = accuracy_score(preds, [1]*len(preds))  # Assuming ground truth is always 1
        precision = precision_score(preds, [1]*len(preds))
        recall = recall_score(preds, [1]*len(preds))
        f1 = f1_score(preds, [1]*len(preds))
        aspect_metrics[aspect] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}

    # Print metrics
    for aspect, metrics_ in aspect_metrics.items():
        print(aspect)
        for metric, value in metrics_.items():
            print(f"{metric}: {value:.2f}")
        print("\n")

# printed for finetuning on 75 datasets

In [13]:
before_metrics=boolean_metrics(true_data,before_finetuned)
print_metrics(before_metrics,mode="before")


The metrics before finetuning is:
Indicators
Accuracy: 0.26
Precision: 0.26
Recall: 1.00
F1-score: 0.42


Description
Accuracy: 0.51
Precision: 0.51
Recall: 1.00
F1-score: 0.68


Geographic_Coverage
Accuracy: 0.51
Precision: 0.51
Recall: 1.00
F1-score: 0.68


Format
Accuracy: 0.06
Precision: 0.06
Recall: 1.00
F1-score: 0.11


Spatial_Resolution
Accuracy: 0.54
Precision: 0.54
Recall: 1.00
F1-score: 0.70


Temporal_Resolution
Accuracy: 0.36
Precision: 0.36
Recall: 1.00
F1-score: 0.53


Temporal_Extent
Accuracy: 0.21
Precision: 0.21
Recall: 1.00
F1-score: 0.34


Latency
Accuracy: 0.53
Precision: 0.53
Recall: 1.00
F1-score: 0.69


Project
Accuracy: 0.29
Precision: 0.29
Recall: 1.00
F1-score: 0.45


Data_Visualization
Accuracy: 0.61
Precision: 0.61
Recall: 1.00
F1-score: 0.76




In [14]:
after_metrics=boolean_metrics(true_data,after_finetuned)
print_metrics(before_metrics,mode="after")

The metrics after finetuning is:
Indicators
Accuracy: 0.26
Precision: 0.26
Recall: 1.00
F1-score: 0.42


Description
Accuracy: 0.51
Precision: 0.51
Recall: 1.00
F1-score: 0.68


Geographic_Coverage
Accuracy: 0.51
Precision: 0.51
Recall: 1.00
F1-score: 0.68


Format
Accuracy: 0.06
Precision: 0.06
Recall: 1.00
F1-score: 0.11


Spatial_Resolution
Accuracy: 0.54
Precision: 0.54
Recall: 1.00
F1-score: 0.70


Temporal_Resolution
Accuracy: 0.36
Precision: 0.36
Recall: 1.00
F1-score: 0.53


Temporal_Extent
Accuracy: 0.21
Precision: 0.21
Recall: 1.00
F1-score: 0.34


Latency
Accuracy: 0.53
Precision: 0.53
Recall: 1.00
F1-score: 0.69


Project
Accuracy: 0.29
Precision: 0.29
Recall: 1.00
F1-score: 0.45


Data_Visualization
Accuracy: 0.61
Precision: 0.61
Recall: 1.00
F1-score: 0.76




In [15]:
print(len(before_finetuned))
print(len(true_data))
print(len(after_finetuned))

72
72
72
